In [1]:
!pip install selenium
from bs4 import BeautifulSoup as bs
import re
import json
import requests as rq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import time
import random
from os import system   
from math import floor
from copy import deepcopy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
%matplotlib inline

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
with open('ESPN_stats.pkl', 'rb') as f:
    espn_dict = pickle.load(f)
with open('new_df.pkl', 'rb') as f:
     new_df = pickle.load(f)
with open('new_df_orig.pkl', 'rb') as f:
    orig_df = pickle.load(f)

In [3]:
for i in espn_dict.items():
    i[1]['year'] = i[0][-4:]

In [4]:
defense_df = pd.concat([i[1] for i in espn_dict.items() if i[0][0] == 'd'])
passing_df = pd.concat([i[1] for i in espn_dict.items() if i[0][0] == 'p'])
rushing_df = pd.concat([i[1] for i in espn_dict.items() if i[0][:2] == 'ru'])
receiving_df = pd.concat([i[1] for i in espn_dict.items() if i[0][:2] == 're'])

In [5]:
print(defense_df.shape)
defense_df.head()

(24550, 13)


,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year
Evan WeaverCAL,LB,103,78,181,2.5,19,3,0,0,0,0,3,2019
Dele HardingILL,LB,68,86,154,0.0,0,0,3,94,54,2,3,2019
Treshaun HaywardWMU,LB,72,70,142,5.0,36,1,0,0,0,0,1,2019
Jahad WoodsWSU,LB,73,68,141,3.0,12,3,1,0,0,0,2,2019
John LakoAKR,LB,60,78,138,1.0,8,2,1,64,64,1,0,2019


In [6]:
print(passing_df.shape)
passing_df.head()

(7456, 12)


,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG,year
Joe BurrowLSU,QB,402,527,76.3,"5,671",10.8,78,60,6,34,202.0,2019
Anthony GordonWSU,QB,493,689,71.6,"5,579",8.1,65,48,16,20,157.9,2019
Cole McDonaldHAW,QB,326,511,63.8,"4,135",8.1,59,33,14,19,147.6,2019
Brady WhiteMEM,QB,269,420,64.0,"4,014",9.6,73,33,11,27,165.0,2019
Brock PurdyISU,QB,312,475,65.7,"3,982",8.4,84,27,9,16,151.1,2019


In [7]:
print(rushing_df.shape)
rushing_df.head()

(22339, 7)


,POS,ATT,YDS,AVG,LNG,TD,year
Chuba HubbardOKST,RB,328,"2,094",6.4,92,21,2019
Malcolm PerryNAVY,QB,295,"2,017",6.8,70,21,2019
J.K. DobbinsOSU,RB,301,"2,003",6.7,68,21,2019
Jonathan TaylorWIS,RB,320,"2,003",6.3,72,21,2019
Jaret PattersonBUFF,RB,312,"1,799",5.8,82,19,2019


In [8]:
print(receiving_df.shape)
receiving_df.head()

(24914, 7)


,POS,REC,YDS,AVG,LNG,TD,year
Ja'Marr ChaseLSU,WR,84,"1,780",21.2,78,20,2019
Omar BaylessARST,WR,93,"1,653",17.8,92,17,2019
Justin JeffersonLSU,WR,111,"1,540",13.9,71,18,2019
Antonio Gandy-GoldenLIB,WR,79,"1,396",17.7,73,10,2019
Devin DuvernayTEX,WR,106,"1,386",13.1,75,9,2019


In [9]:
#defense name and school splitting
schools = []
names = []
expression = r"[A-Z]{2,}"
multiple_matches = []
checker = []
for i in defense_df.index:
    j = i.split()[-1]
    matches = re.finditer(expression, j)
    for matchNum, match in enumerate(matches, start=0):
        if matchNum >= 1:
            multiple_matches.append(i)
            break
        schools.append(match.group())
        names.append(i[:-len(match.group())])
        checker.append(i)
    if (i not in checker) & (i not in multiple_matches):
            schools.append('none')
            names.append(i)
defense_df['college'] = schools
split_names = [i.split() for i in names]
all_names = []
for i in split_names:
    temp = []
    for j in i:
        temp.append(j.replace('.', ''))
    all_names.append(' '.join(temp).lower())
defense_df['search_name'] = all_names
defense_df.set_index(defense_df['search_name'], inplace=True)
defense_df.drop(columns=['search_name'], inplace=True)
print(len(names))
print(len(schools))
print(len(defense_df.index))

#passing name and school splitting
schools = []
names = []
expression = r"[A-Z]{2,}"
multiple_matches = []
checker = []
for i in passing_df.index:
    j = i.split()[-1]
    matches = re.finditer(expression, j)
    for matchNum, match in enumerate(matches, start=0):
        if matchNum >= 1:
            multiple_matches.append(i)
            break
        schools.append(match.group())
        names.append(i[:-len(match.group())])
        checker.append(i)
    if (i not in checker) & (i not in multiple_matches):
            schools.append('none')
            names.append(i)
passing_df['college'] = schools
split_names = [i.split() for i in names]
all_names = []
for i in split_names:
    temp = []
    for j in i:
        temp.append(j.replace('.', ''))
    all_names.append(' '.join(temp).lower())
passing_df['search_name'] = all_names
passing_df.set_index(passing_df['search_name'], inplace=True)
passing_df.drop(columns=['search_name'], inplace=True)
print(len(names))
print(len(schools))
print(len(passing_df.index))

#rushing name and school splitting
schools = []
names = []
expression = r"[A-Z]{2,}"
multiple_matches = []
checker = []
for i in rushing_df.index:
    j = i.split()[-1]
    matches = re.finditer(expression, j)
    for matchNum, match in enumerate(matches, start=0):
        if matchNum >= 1:
            multiple_matches.append(i)
            break
        schools.append(match.group())
        names.append(i[:-len(match.group())])
        checker.append(i)
    if (i not in checker) & (i not in multiple_matches):
            schools.append('none')
            names.append(i)
rushing_df['college'] = schools
split_names = [i.split() for i in names]
all_names = []
for i in split_names:
    temp = []
    for j in i:
        temp.append(j.replace('.', ''))
    all_names.append(' '.join(temp).lower())
rushing_df['search_name'] = all_names
rushing_df.set_index(rushing_df['search_name'], inplace=True)
rushing_df.drop(columns=['search_name'], inplace=True)
print(len(names))
print(len(schools))
print(len(rushing_df.index))

#receiving name and school splitting
schools = []
names = []
expression = r"[A-Z]{2,}"
multiple_matches = []
checker = []
for i in receiving_df.index:
    j = i.split()[-1]
    matches = re.finditer(expression, j)
    for matchNum, match in enumerate(matches, start=0):
        if matchNum >= 1:
            multiple_matches.append(i)
            break
        schools.append(match.group())
        names.append(i[:-len(match.group())])
        checker.append(i)
    if (i not in checker) & (i not in multiple_matches):
            schools.append('none')
            names.append(i)
receiving_df['college'] = schools
split_names = [i.split() for i in names]
all_names = []
for i in split_names:
    temp = []
    for j in i:
        temp.append(j.replace('.', ''))
    all_names.append(' '.join(temp).lower())
receiving_df['search_name'] = all_names
receiving_df.set_index(receiving_df['search_name'], inplace=True)
receiving_df.drop(columns=['search_name'], inplace=True)
print(len(names))
print(len(schools))
print(len(receiving_df.index))

24550
24550
24550
7456
7456
7456
22339
22339
22339
24914
24914
24914


In [10]:
defense_df.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college
search_name,,,,,,,,,,,,,,
evan weaver,LB,103,78,181,2.5,19,3,0,0,0,0,3,2019,CAL
dele harding,LB,68,86,154,0.0,0,0,3,94,54,2,3,2019,ILL
treshaun hayward,LB,72,70,142,5.0,36,1,0,0,0,0,1,2019,WMU
jahad woods,LB,73,68,141,3.0,12,3,1,0,0,0,2,2019,WSU
john lako,LB,60,78,138,1.0,8,2,1,64,64,1,0,2019,AKR


In [11]:
passing_df.head()

,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG,year,college
search_name,,,,,,,,,,,,,
joe burrow,QB,402,527,76.3,"5,671",10.8,78,60,6,34,202.0,2019,LSU
anthony gordon,QB,493,689,71.6,"5,579",8.1,65,48,16,20,157.9,2019,WSU
cole mcdonald,QB,326,511,63.8,"4,135",8.1,59,33,14,19,147.6,2019,HAW
brady white,QB,269,420,64.0,"4,014",9.6,73,33,11,27,165.0,2019,MEM
brock purdy,QB,312,475,65.7,"3,982",8.4,84,27,9,16,151.1,2019,ISU


In [12]:
rushing_df.head()

,POS,ATT,YDS,AVG,LNG,TD,year,college
search_name,,,,,,,,
chuba hubbard,RB,328,"2,094",6.4,92,21,2019,OKST
malcolm perry,QB,295,"2,017",6.8,70,21,2019,NAVY
jk dobbins,RB,301,"2,003",6.7,68,21,2019,OSU
jonathan taylor,RB,320,"2,003",6.3,72,21,2019,WIS
jaret patterson,RB,312,"1,799",5.8,82,19,2019,BUFF


In [13]:
receiving_df.head()

,POS,REC,YDS,AVG,LNG,TD,year,college
search_name,,,,,,,,
ja'marr chase,WR,84,"1,780",21.2,78,20,2019,LSU
omar bayless,WR,93,"1,653",17.8,92,17,2019,ARST
justin jefferson,WR,111,"1,540",13.9,71,18,2019,LSU
antonio gandy-golden,WR,79,"1,396",17.7,73,10,2019,LIB
devin duvernay,WR,106,"1,386",13.1,75,9,2019,TEX


In [14]:
#create a new dataframe from new_df that contains only the players in defense df and combine df
defense_only = pd.DataFrame(columns=defense_df.columns)
for i in new_df['search_name']:
    if i in defense_df.index:
        try:
            temp_df = pd.DataFrame(data = defense_df.loc[i])
            defense_only  = pd.concat([defense_only, temp_df.T])
        except:
            temp_df = pd.DataFrame(data = defense_df.loc[i])
            defense_only  = pd.concat([defense_only, temp_df])
print(len(defense_only.index))
print(len(set(defense_only.index)))

#create a new dataframe from new_df that contains only the players in passing df and combine df
passing_only = pd.DataFrame(columns=passing_df.columns)
for i in new_df['search_name']:
    if i in passing_df.index:
        try:
            temp_df = pd.DataFrame(data = passing_df.loc[i])
            passing_only  = pd.concat([passing_only, temp_df.T])
        except:
            temp_df = pd.DataFrame(data = passing_df.loc[i])
            passing_only  = pd.concat([passing_only, temp_df])
print(len(passing_only.index))
print(len(set(passing_only.index)))

#create a new dataframe from new_df that contains only the players in rushing df and combine df
rushing_only = pd.DataFrame(columns=rushing_df.columns)
for i in new_df['search_name']:
    if i in rushing_df.index:
        try:
            temp_df = pd.DataFrame(data = rushing_df.loc[i])
            rushing_only  = pd.concat([rushing_only, temp_df.T])
        except:
            temp_df = pd.DataFrame(data = rushing_df.loc[i])
            rushing_only  = pd.concat([rushing_only, temp_df])
print(len(rushing_only.index))
print(len(set(rushing_only.index)))

#create a new dataframe from new_df that contains only the players in receiving df and combine df
receiving_only = pd.DataFrame(columns=receiving_df.columns)
for i in new_df['search_name']:
    if i in receiving_df.index:
        try:
            temp_df = pd.DataFrame(data = receiving_df.loc[i])
            receiving_only  = pd.concat([receiving_only, temp_df.T])
        except:
            temp_df = pd.DataFrame(data = receiving_df.loc[i])
            receiving_only  = pd.concat([receiving_only, temp_df])
print(len(receiving_only.index))
print(len(set(receiving_only.index)))

5252
2046
1499
634
4049
1476
4548
1577


In [15]:
defense_only.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college
brian allen,CB,27,8,35,1.0,4,6,4,0,0,0,0,2016,UTAH
kyle fuller,CB,17,7,24,0.0,0,0,2,8,11,0,1,2013,VT
kyle fuller,CB,44,8,52,0.0,0,0,2,0,0,0,1,2012,VT
kyle fuller,CB,46,19,65,4.5,-39,0,2,2,2,0,0,2011,VT
kyle fuller,CB,46,19,65,4.5,-39,0,2,2,2,0,0,2011,VT


In [16]:
#Cast all stats in all dfs to floats

defense_only = defense_only.astype({'SOLO':'float',
                                'AST':'float',
                                'TOT':'float',
                                'SACK':'float',
                                'YDS':'float',
                                'PD':'float',
                                'INT':'float',
                                'YDS':'float',
                                'LNG':'float',
                                'TD':'float',
                                'FF':'float',})
comma_free = []
for i in passing_only['YDS']:
    comma_free.append(i.replace(',', ''))
passing_only['YDS'] = comma_free 
passing_only = passing_only.astype({'CMP':'float',
                                'ATT':'float',
                                'CMP%':'float',
                                'YDS':'float',
                                'AVG':'float',
                                'LNG':'float',
                                'TD':'float',
                                'INT':'float',
                                'SACK':'float',
                                'RTG':'float',})
comma_free = []
for i in rushing_only['YDS']:
    comma_free.append(i.replace(',', ''))
rushing_only['YDS'] = comma_free
rushing_only = rushing_only.astype({'ATT':'float',
                                'YDS':'float',
                                'AVG':'float',
                                'LNG':'float',
                                'TD':'float',})
comma_free = []
for i in receiving_only['YDS']:
    comma_free.append(i.replace(',', ''))
receiving_only['YDS'] = comma_free
receiving_only = receiving_only.astype({'REC':'float',
                                'YDS':'float',
                                'AVG':'float',
                                'LNG':'float',
                                'TD':'float',})

In [17]:
defense_only.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college
brian allen,CB,27.0,8.0,35.0,1.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,2016,UTAH
kyle fuller,CB,17.0,7.0,24.0,0.0,0.0,0.0,2.0,8.0,11.0,0.0,1.0,2013,VT
kyle fuller,CB,44.0,8.0,52.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2012,VT
kyle fuller,CB,46.0,19.0,65.0,4.5,-39.0,0.0,2.0,2.0,2.0,0.0,0.0,2011,VT
kyle fuller,CB,46.0,19.0,65.0,4.5,-39.0,0.0,2.0,2.0,2.0,0.0,0.0,2011,VT


In [18]:
passing_only.head()

,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG,year,college
nick nelson,QB,3.0,6.0,50.0,22.0,3.7,10.0,0.0,0.0,2.0,80.8,2007,COLO
josh jackson,QB,98.0,207.0,47.3,1274.0,6.2,80.0,12.0,6.0,25.0,112.4,2019,MD
josh jackson,QB,36.0,58.0,62.1,575.0,9.9,72.0,5.0,1.0,6.0,170.3,2018,VT
josh jackson,QB,236.0,396.0,59.6,2991.0,7.6,70.0,20.0,9.0,22.0,135.2,2017,VT
chris campbell,QB,27.0,55.0,49.1,322.0,5.9,31.0,3.0,2.0,3.0,109.0,2012,USM


In [19]:
rushing_only.head()

,POS,ATT,YDS,AVG,LNG,TD,year,college
tyler orlosky,C,1.0,-2.0,-2.0,0.0,0.0,2016,WVU
jack allen,C,1.0,9.0,9.0,9.0,1.0,2015,MSU
rodney hudson,G,0.0,9.0,0.0,9.0,0.0,2009,FSU
jason brown,WR,1.0,1.0,1.0,1.0,0.0,2005,AFA
jamal peters,CB,1.0,8.0,8.0,8.0,0.0,2017,MSST


In [20]:
receiving_only.head()

,POS,REC,YDS,AVG,LNG,TD,year,college
garth gerhart,C,1.0,1.0,1.0,1.0,0.0,2010,ASU
aq shipley,C,1.0,4.0,4.0,4.0,0.0,2007,PSU
jason brown,WR,49.0,874.0,17.8,61.0,5.0,2005,AFA
jason brown,WR,9.0,164.0,18.2,45.0,1.0,2004,AFA
justin layne,CB,1.0,11.0,11.0,11.0,0.0,2018,MSU


In [21]:
# remove duplicate years,such that all remaining years are unique for each player

defense_only['name'] = defense_only.index + defense_only.college
career_defense = pd.DataFrame(columns=defense_only.columns)
for j in set(defense_only['name']):
    temp = defense_only.loc[(defense_only['name'] == j)]
    unique_years= []
    unique_seasons = pd.DataFrame(data = temp.iloc[0], columns=defense_only.columns)
    for i in temp['year']:
        if i not in unique_years:
            unique_years.append(i)
            temp_df = pd.DataFrame(temp.loc[temp['year'] == i,].iloc[0])
            unique_seasons = pd.concat([unique_seasons, temp_df.T])
    career_defense = pd.concat([career_defense,unique_seasons])

passing_only['name'] = passing_only.index + passing_only.college
career_passing = pd.DataFrame(columns=passing_only.columns)
for j in set(passing_only['name']):
    temp = passing_only.loc[(passing_only['name'] == j)]
    unique_years= []
    unique_seasons = pd.DataFrame(data = temp.iloc[0], columns=passing_only.columns)
    for i in temp['year']:
        if i not in unique_years:
            unique_years.append(i)
            temp_df = pd.DataFrame(temp.loc[temp['year'] == i,].iloc[0])
            unique_seasons = pd.concat([unique_seasons, temp_df.T])
    career_passing = pd.concat([career_passing,unique_seasons])    

rushing_only['name'] = rushing_only.index + rushing_only.college
career_rushing = pd.DataFrame(columns=rushing_only.columns)
for j in set(rushing_only['name']):
    temp = rushing_only.loc[(rushing_only['name'] == j)]
    unique_years= []
    unique_seasons = pd.DataFrame(data = temp.iloc[0], columns=rushing_only.columns)
    for i in temp['year']:
        if i not in unique_years:
            unique_years.append(i)
            temp_df = pd.DataFrame(temp.loc[temp['year'] == i,].iloc[0])
            unique_seasons = pd.concat([unique_seasons, temp_df.T])
    career_rushing = pd.concat([career_rushing,unique_seasons])

receiving_only['name'] = receiving_only.index + receiving_only.college
career_receiving = pd.DataFrame(columns=receiving_only.columns)
for j in set(receiving_only['name']):
    temp = receiving_only.loc[(receiving_only['name'] == j)]
    unique_years= []
    unique_seasons = pd.DataFrame(data = temp.iloc[0], columns=receiving_only.columns)
    for i in temp['year']:
        if i not in unique_years:
            unique_years.append(i)
            temp_df = pd.DataFrame(temp.loc[temp['year'] == i,].iloc[0])
            unique_seasons = pd.concat([unique_seasons, temp_df.T])
    career_receiving = pd.concat([career_receiving,unique_seasons])

In [22]:
career_defense.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college,name
charles alexander,DT,10,18,28,2,-9,0,0,0,0,0,0,2006,LSU,charles alexanderLSU
micah johnson,LB,51,54,105,1,-1,0,0,0,0,0,0,2009,UK,micah johnsonUK
micah johnson,LB,57,30,87,2,-9,0,0,0,0,0,1,2008,UK,micah johnsonUK
micah johnson,LB,14,15,29,0,0,0,0,0,0,0,1,2006,UK,micah johnsonUK
charles harris,LB,34,15,49,0.5,-5,0,0,0,0,0,0,2014,MEM,charles harrisMEM


In [23]:
career_passing.head()

,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG,year,college,name
cam newton,QB,185,280,66.1,2854,10.2,94,30,7,23,182,2010,AUB,cam newtonAUB
derron smith,S,0,1,0,0,0,0,0,0,0,0,2013,FRES,derron smithFRES
rodney adams,WR,0,1,0,0,0,0,0,0,0,0,2014,USF,rodney adamsUSF
quinton ganther,RB,0,1,0,0,0,0,0,0,0,0,2005,UTAH,quinton gantherUTAH
quinton ganther,RB,0,1,0,0,0,0,0,0,0,0,2004,UTAH,quinton gantherUTAH


In [24]:
career_rushing.head()

,POS,ATT,YDS,AVG,LNG,TD,year,college,name
micah johnson,LB,2,1,0.5,1,1,2006,UK,micah johnsonUK
ej manuel,QB,103,310,3,28,4,2012,FSU,ej manuelFSU
ej manuel,QB,110,151,1.4,28,4,2011,FSU,ej manuelFSU
ej manuel,QB,41,170,4.1,32,1,2010,FSU,ej manuelFSU
ej manuel,QB,44,196,4.5,24,2,2009,FSU,ej manuelFSU


In [25]:
career_receiving.head()

,POS,REC,YDS,AVG,LNG,TD,year,college,name
cedric benson,RB,22,179,8.1,20,1,2004,TEX,cedric bensonTEX
david reed,WR,81,1188,14.7,90,5,2009,UTAH,david reedUTAH
david reed,WR,25,427,17.1,47,6,2008,UTAH,david reedUTAH
tim cornett,RB,30,164,5.5,20,0,2013,UNLV,tim cornettUNLV
tim cornett,RB,10,54,5.4,20,0,2011,UNLV,tim cornettUNLV


In [26]:
career_totals_defense = pd.DataFrame(data=None, columns=career_defense.columns)
for i in set(career_defense['name']):
    matches = career_defense.loc[career_defense['name']==i]
    temp_df = pd.DataFrame(matches)
    temp_sum = temp_df.iloc[0]
    
    for j in range(1,4):
        try:
            temp_sum = temp_sum + temp_df.iloc[j]
            temp_sum[0] = temp_sum[0][len(temp_df.iloc[1,0]):]
            temp_sum[-1] = temp_sum[-1][len(temp_df.iloc[1,-1]):]
            temp_sum[-2] = temp_sum[-2][:len(temp_df.iloc[1,-2])]
            temp_sum[-3] = (int(temp_sum[-3][:len(temp_df.iloc[1,-3])]) + 1)
        except:
            break
    temp_sum_df = pd.DataFrame(temp_sum).T
    career_totals_defense = pd.concat([career_totals_defense, temp_sum_df])


career_totals_passing = pd.DataFrame(data=None, columns=career_passing.columns)
for i in set(career_passing['name']):
    matches = career_passing.loc[career_passing['name']==i]
    temp_df = pd.DataFrame(matches)
    temp_sum = temp_df.iloc[0]
    
    for j in range(1,4):
        try:
            temp_sum = temp_sum + temp_df.iloc[j]
            temp_sum[0] = temp_sum[0][len(temp_df.iloc[1,0]):]
            temp_sum[-1] = temp_sum[-1][len(temp_df.iloc[1,-1]):]
            temp_sum[-2] = temp_sum[-2][:len(temp_df.iloc[1,-2])]
            temp_sum[-3] = (int(temp_sum[-3][:len(temp_df.iloc[1,-3])]) + 1)
        except:
            break
    temp_sum_df = pd.DataFrame(temp_sum).T
    career_totals_passing = pd.concat([career_totals_passing, temp_sum_df])


career_totals_rushing = pd.DataFrame(data=None, columns=career_rushing.columns)
for i in set(career_rushing['name']):
    matches = career_rushing.loc[career_rushing['name']==i]
    temp_df = pd.DataFrame(matches)
    temp_sum = temp_df.iloc[0]
    
    for j in range(1,4):
        try:
            temp_sum = temp_sum + temp_df.iloc[j]
            temp_sum[0] = temp_sum[0][len(temp_df.iloc[1,0]):]
            temp_sum[-1] = temp_sum[-1][len(temp_df.iloc[1,-1]):]
            temp_sum[-2] = temp_sum[-2][:len(temp_df.iloc[1,-2])]
            temp_sum[-3] = (int(temp_sum[-3][:len(temp_df.iloc[1,-3])]) + 1)
        except:
            break
    temp_sum_df = pd.DataFrame(temp_sum).T
    career_totals_rushing = pd.concat([career_totals_rushing, temp_sum_df])


career_totals_receiving = pd.DataFrame(data=None, columns=career_receiving.columns)
for i in set(career_receiving['name']):
    matches = career_receiving.loc[career_receiving['name']==i]
    temp_df = pd.DataFrame(matches)
    temp_sum = temp_df.iloc[0]
    
    for j in range(1,4):
        try:
            temp_sum = temp_sum + temp_df.iloc[j]
            temp_sum[0] = temp_sum[0][len(temp_df.iloc[1,0]):]
            temp_sum[-1] = temp_sum[-1][len(temp_df.iloc[1,-1]):]
            temp_sum[-2] = temp_sum[-2][:len(temp_df.iloc[1,-2])]
            temp_sum[-3] = (int(temp_sum[-3][:len(temp_df.iloc[1,-3])]) + 1)
        except:
            break
    temp_sum_df = pd.DataFrame(temp_sum).T
    career_totals_receiving = pd.concat([career_totals_receiving, temp_sum_df])

In [27]:
career_totals_defense.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college,name
charles alexander,DT,10,18,28,2,-9,0,0,0,0,0,0,2006,LSU,charles alexanderLSU
micah johnson,LB,108,84,192,3,-10,0,0,0,0,0,1,2010,UK,micah johnsonUK
charles harris,LB,77,44,121,0.5,-5,0,0,0,0,0,0,2015,MEM,charles harrisMEM
marcus maye,S,75,54,129,1,10,6,3,6,6,0,5,2017,FLA,marcus mayeFLA
chris mckenzie,CB,35,7,42,0,-5,0,1,0,0,0,0,2004,ASU,chris mckenzieASU


In [28]:
career_totals_passing.head()

,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG,year,college,name
cam newton,QB,185,280,66.1,2854,10.2,94,30,7,23,182,2010,AUB,cam newtonAUB
derron smith,S,0,1,0,0,0,0,0,0,0,0,2013,FRES,derron smithFRES
rodney adams,WR,0,1,0,0,0,0,0,0,0,0,2014,USF,rodney adamsUSF
quinton ganther,RB,0,2,0,0,0,0,0,0,0,0,2006,UTAH,quinton gantherUTAH
adrian arrington,WR,1,2,50,11,5.5,11,1,0,0,261.2,2007,MICH,adrian arringtonMICH


In [29]:
career_totals_rushing.head()

,POS,ATT,YDS,AVG,LNG,TD,year,college,name
micah johnson,LB,2,1,0.5,1,1,2006,UK,micah johnsonUK
ej manuel,QB,213,461,4.4,56,8,2013,FSU,ej manuelFSU
cedric benson,RB,326,1834,5.6,60,19,2004,TEX,cedric bensonTEX
david reed,WR,5,19,16,25,1,2010,UTAH,david reedUTAH
tim cornett,RB,506,2516,10,151,22,2014,UNLV,tim cornettUNLV


In [30]:
career_totals_receiving.head()

,POS,REC,YDS,AVG,LNG,TD,year,college,name
cedric benson,RB,22,179,8.1,20,1,2004,TEX,cedric bensonTEX
david reed,WR,106,1615,31.8,137,11,2010,UTAH,david reedUTAH
damarius bilbo,WR,44,624,23.1,78,2,2006,GT,damarius bilboGT
tim cornett,RB,40,218,10.9,40,0,2014,UNLV,tim cornettUNLV
james starks,RB,93,672,14.5,139,3,2009,BUFF,james starksBUFF


In [31]:
new_df.drop(columns=['stats'], inplace=True)
new_df.drop(columns=['Since', 'Until'], inplace=True)
new_df.reset_index(inplace=True)
new_df.set_index('search_name', inplace=True)
new_df.drop(columns=['Player'], inplace=True)

In [32]:
new_df.head()

,Pos,Year,Age,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Round
search_name,,,,,,,,,,,,,
sean welsh,C,2018,23.0,Iowa,6-3,306,5.430000,25.0000,20.00000,103.000000,7.90000,4.81000,8
coleman shelton,C,2018,22.0,Washington,6-3,292,5.240000,26.5000,23.64615,110.000000,7.62000,4.59000,8
frank ragnow,C,2018,21.0,Arkansas,6-5,312,5.319924,26.5097,26.00000,98.421912,7.86268,4.72272,1
scott quessenberry,C,2018,22.0,UCLA,6-4,310,5.090000,33.5000,25.00000,111.000000,7.50000,4.69000,5
billy price,C,2018,23.0,Ohio St.,6-4,305,5.319924,26.5097,23.64615,98.421912,7.86268,4.72272,1


In [33]:
career_totals_defense.head()

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college,name
charles alexander,DT,10,18,28,2,-9,0,0,0,0,0,0,2006,LSU,charles alexanderLSU
micah johnson,LB,108,84,192,3,-10,0,0,0,0,0,1,2010,UK,micah johnsonUK
charles harris,LB,77,44,121,0.5,-5,0,0,0,0,0,0,2015,MEM,charles harrisMEM
marcus maye,S,75,54,129,1,10,6,3,6,6,0,5,2017,FLA,marcus mayeFLA
chris mckenzie,CB,35,7,42,0,-5,0,1,0,0,0,0,2004,ASU,chris mckenzieASU


In [34]:
set(career_totals_defense.POS)

{'-',
 'B',
 'CB',
 'DB',
 'DE',
 'DL',
 'DT',
 'FB',
 'FS',
 'LB',
 'NT',
 'OT',
 'P',
 'PK',
 'QB',
 'RB',
 'S',
 'T',
 'TE',
 'WR'}

In [35]:
set(new_df.Pos)

{'C',
 'CB',
 'DE',
 'DT',
 'FB',
 'FS',
 'ILB',
 'OG',
 'OLB',
 'OT',
 'QB',
 'RB',
 'SS',
 'TE',
 'WR'}

In [36]:
new_df['search_pos'] = new_df['Pos']

In [37]:
new_df.head()

,Pos,Year,Age,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Round,search_pos
search_name,,,,,,,,,,,,,,
sean welsh,C,2018,23.0,Iowa,6-3,306,5.430000,25.0000,20.00000,103.000000,7.90000,4.81000,8,C
coleman shelton,C,2018,22.0,Washington,6-3,292,5.240000,26.5000,23.64615,110.000000,7.62000,4.59000,8,C
frank ragnow,C,2018,21.0,Arkansas,6-5,312,5.319924,26.5097,26.00000,98.421912,7.86268,4.72272,1,C
scott quessenberry,C,2018,22.0,UCLA,6-4,310,5.090000,33.5000,25.00000,111.000000,7.50000,4.69000,5,C
billy price,C,2018,23.0,Ohio St.,6-4,305,5.319924,26.5097,23.64615,98.421912,7.86268,4.72272,1,C


In [38]:
career_totals_defense.drop(columns=['name'], inplace=True)
career_totals_passing.drop(columns=['name'], inplace=True)
career_totals_rushing.drop(columns=['name'], inplace=True)
career_totals_receiving.drop(columns=['name'], inplace=True)

In [39]:
%%timeit

college_first_letters = []
for i in new_df.School:
   
    split_college = i.split()
    
    if split_college[-1] == 'St.':
        split_college.append('University')
    
    joined_college = ''
    
    for j in split_college:

        if len(split_college) == 1:
            joined_college += j[:3]
        else:
            joined_college += j[0]        
    college_first_letters.append(joined_college.upper())
new_df['College'] = college_first_letters

4 ms ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [772]:
for i in career_totals_defense.index:
    for j in new_df.index:
        if i == j:
            print(i,j)
            if type(career_totals_defense.loc[i,'POS']) is str and type(new_df.loc[j,'search_pos']) is str:
        
                print(type(career_totals_defense.loc[i,'POS']),type(new_df.loc[j,'search_pos']))
                
                
                print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'])
                print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'])
                find = career_totals_defense.loc[i,'POS'].find(new_df.loc[j, 'search_pos'])
                print('find value {}, type {}'.format(find, type(find)))
                if new_df.loc[j, 'search_pos'].find(career_totals_defense.loc[i,'POS']) != -1:
                    print('hooray')
                    print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'])
                    print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'])
                    print('----')
                else:
                    print('not hooray')
                    print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'])
                    print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'])
                    print('----')
                    if career_totals_defense.loc[i, 'college'][0][:2] == new_df.loc[i, 'College'][0][:2]:
                        try:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j]).T
                            k = pd.DataFrame(career_totals_defense.loc[i]).T
                            z = pd.concat([h,k], axis=1)
                        except:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j])
                            k = pd.DataFrame(career_totals_defense.loc[i])
                            z = pd.concat([h,k], axis=1)
                            
            elif type(career_totals_defense.loc[i,'POS']) is not str and type(new_df.loc[j,'search_pos']) is str:
                
                print(type(career_totals_defense.loc[i,'POS']),type(new_df.loc[j,'search_pos']))
                
                #for x in career_totals_defense.loc[i,'POS']:
                    
                print(type(career_totals_defense.loc[i,'POS']))
                
                if career_totals_defense.loc[i,'POS'] == new_df.loc[j, 'search_pos']:
                    
                    print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'][0])
                    print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'][0])
                    
                    if career_totals_defense.loc[i, 'college'][0][:2] == new_df.loc[i, 'College'][0][:2]:
                        try:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j]).T
                            k = pd.DataFrame(career_totals_defense.loc[i]).T
                            z = pd.concat([h,k], axis=1)
                        except:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j])
                            k = pd.DataFrame(career_totals_defense.loc[i])
                            z = pd.concat([h,k], axis=1)
                    
            elif type(career_totals_defense.loc[i,'POS']) is str and type(new_df.loc[j,'search_pos']) is not str:
                
                print(type(career_totals_defense.loc[i,'POS']),type(new_df.loc[j,'search_pos']))
                
                #for x in career_totals_defense.loc[i,'POS']:
                    
                print(type(career_totals_defense.loc[i,'POS']))
                
                if career_totals_defense.loc[i,'POS'] == new_df.loc[j, 'search_pos']:
                    
                    print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'][0])
                    print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'][0])
                    
                    if career_totals_defense.loc[i, 'college'][0][:2] == new_df.loc[i, 'College'][0][:2]:
                        try:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j]).T
                            k = pd.DataFrame(career_totals_defense.loc[i]).T
                            z = pd.concat([h,k], axis=1)
                        except:
                            print('success')
                            h = pd.DataFrame(new_df.loc[j])
                            k = pd.DataFrame(career_totals_defense.loc[i])
                            z = pd.concat([h,k], axis=1)
                            
            elif type(career_totals_defense.loc[i,'POS']) is not str and type(new_df.loc[j,'search_pos']) is not str:
                
                print(type(career_totals_defense.loc[i,'POS']), type(new_df.loc[j,'search_pos']))
                
                #for x in career_totals_defense.loc[i,'POS']:
                    
                print(type(career_totals_defense.loc[i,'POS']))
                
                if career_totals_defense.loc[i,'POS'] == new_df.loc[j, 'search_pos']:
                    
                    print(i, career_totals_defense.loc[i,'college'], career_totals_defense.loc[i,'POS'][0])
                    print(j, new_df.loc[j, 'College'], new_df.loc[j, 'search_pos'][0])
                    
                    if career_totals_defense.loc[i, 'college'][0][:2] == new_df.loc[i, 'College'][0][:2]:
                        try:
                            h = pd.DataFrame(new_df.loc[j]).T
                            k = pd.DataFrame(career_totals_defense.loc[i]).T
                            z = pd.concat([h,k], axis=1)
                        except:
                            h = pd.DataFrame(new_df.loc[j])
                            k = pd.DataFrame(career_totals_defense.loc[i])
                            z = pd.concat([h,k], axis=1)

quan sturdivant quan sturdivant
<class 'str'> <class 'str'>
quan sturdivant UNC LB
quan sturdivant NC ILB
find value -1, type <class 'int'>
hooray
quan sturdivant UNC LB
quan sturdivant NC ILB
----
brodney pool brodney pool
<class 'str'> <class 'str'>
brodney pool OKLA S
brodney pool OKL FS
find value -1, type <class 'int'>
hooray
brodney pool OKLA S
brodney pool OKL FS
----
scott starks scott starks
<class 'str'> <class 'str'>
scott starks WIS DB
scott starks WIS CB
find value -1, type <class 'int'>
not hooray
scott starks WIS DB
scott starks WIS CB
----
success
zach cunningham zach cunningham
<class 'str'> <class 'str'>
zach cunningham VAN LB
zach cunningham VAN OLB
find value -1, type <class 'int'>
hooray
zach cunningham VAN LB
zach cunningham VAN OLB
----
clifton ryan clifton ryan
<class 'str'> <class 'str'>
clifton ryan MSU DL
clifton ryan MSU DT
find value -1, type <class 'int'>
not hooray
clifton ryan MSU DL
clifton ryan MSU DT
----
success
chas alecxih chas alecxih
<class 'str'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [745]:
h

,Pos,Year,Age,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Round,search_pos,College
ryan glasgow,DT,2017,23,Michigan,6-3,302,5.13,27.5,20,105,7.55,4.5,4,DT,MIC


In [746]:
h.T

,ryan glasgow
Pos,DT
Year,2017
Age,23
School,Michigan
Height,6-3
Wt,302
40YD,5.13
Vertical,27.5
BenchReps,20
Broad Jump,105


In [738]:
k.T

,chris hawkins,chris hawkins,chris hawkins
POS,CB,S,DB
SOLO,21,91,15
AST,29,66,12
TOT,50,157,27
SACK,0,3,0
YDS,0,22,0
PD,0,2,0
INT,3,3,0
YDS,-13,27,0
LNG,0,27,0


In [741]:
cat = pd.concat([h,k], axis=1)

ValueError: Shape of passed values is (2, 29), indices imply (1, 29)

In [747]:
career_totals_defense.loc['brandon marshall']

,POS,SOLO,AST,TOT,SACK,YDS,PD,INT,YDS,LNG,TD,FF,year,college
brandon marshall,WR,28,19,47,0,-4,0,1,-4,0,0,1,2004,UCF
brandon marshall,LB,85,72,157,3,-18,0,2,0,0,0,2,2012,NEV


In [748]:
new_df.loc['brandon marshall']

,Pos,Year,Age,School,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Round,search_pos,College
search_name,,,,,,,,,,,,,,,
brandon marshall,OLB,2012,22.0,Nevada,6-1,242,4.72,30.5,28.000000,108.0,7.050000,4.090000,5,OLB,NEV
brandon marshall,WR,2006,21.0,Central Florida,6-4,229,4.52,37.0,12.486491,120.0,7.079518,4.294111,4,WR,CF


In [749]:
career_totals_defense.loc[i,'POS']

brandon marshall    WR
brandon marshall    LB
Name: POS, dtype: object

In [751]:
new_df.loc[j,'search_pos']

search_name
brandon marshall    OLB
brandon marshall     WR
Name: search_pos, dtype: object